## Linear Regression with PyTorch

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

In [1]:
# Uncomment the command below if Numpy or PyTorch is not installed
# !conda install numpy pytorch cpuonly -c pytorch -y

In [2]:
import numpy as np
import torch

## Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [3]:
# Input (temperature, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]],dtype='float32')

In [4]:
#Target (apple, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]],dtype='float32')

In [5]:
#Convert inputs and targets into tensor
train_data = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [6]:
#inputs tensor
print(inputs)

[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]


In [7]:
#targets tensor
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [8]:
#create random weights and biases
weights = torch.randn(2, 3, requires_grad=True)
biases = torch.randn(2, requires_grad=True)

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

In [9]:
#weights
print(weights)

tensor([[ 0.0778,  0.3608,  0.7136],
        [-0.5901, -1.4620, -0.4219]], requires_grad=True)


In [10]:
#biases
print(biases)

tensor([ 0.6570, -1.4724], requires_grad=True)


In [11]:
#define model
def model(train_data, weights, biases):
    return train_data @ weights.t() + biases   

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [12]:
#Generate predictions
predictions = model(train_data, weights, biases)
print(predictions)

tensor([[  61.1897, -160.6452],
        [  85.1513, -210.8294],
        [  97.1534, -273.1904],
        [  50.5048, -140.1366],
        [  90.6085, -212.0761]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [13]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`predictions` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [14]:
# MSE loss error
def mse(predictions,targets):
    diff = predictions - targets
    return torch.sum(diff * diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [15]:
loss = mse(predictions,targets)
print(loss)

tensor(45790.1758, grad_fn=<DivBackward0>)


 Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [16]:
#compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [17]:
#gradients for weights
print(weights)
print(weights.grad)

tensor([[ 0.0778,  0.3608,  0.7136],
        [-0.5901, -1.4620, -0.4219]], requires_grad=True)
tensor([[   181.6881,   -435.6598,   -118.1979],
        [-24292.8652, -27344.7852, -16632.6504]])



The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [18]:
weights.grad.zero_()
biases.grad.zero_()
print(weights.grad)
print(biases.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [19]:
#Generate predictions
predictions = model(train_data,weights,biases)
print(predictions)

tensor([[  61.1897, -160.6452],
        [  85.1513, -210.8294],
        [  97.1534, -273.1904],
        [  50.5048, -140.1366],
        [  90.6085, -212.0761]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [20]:
#Calculate the loss
loss = mse(predictions,targets)
print(loss)

tensor(45790.1758, grad_fn=<DivBackward0>)


In [21]:
#Compute gradients
loss.backward()
print(weights.grad)
print(biases.grad)

tensor([[   181.6881,   -435.6598,   -118.1979],
        [-24292.8652, -27344.7852, -16632.6504]])
tensor([   0.7215, -291.3755])


Finally, we update the weights and biases using the gradients computed above.

In [22]:
# Adjust weights and reset gradients
with torch.no_grad():
    weights -= weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    weights.grad.zero_()
    biases.grad.zero_()

A few things to note above:

* We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

* We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the *learning rate* of the algorithm. 

* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [23]:
print(weights)
print(biases)

tensor([[ 0.0759,  0.3651,  0.7148],
        [-0.3471, -1.1886, -0.2556]], requires_grad=True)
tensor([ 0.6570, -1.4695], requires_grad=True)


With the new weights and biases, the model should have low loss

In [24]:
#Calculate loss
predictions = model(train_data,weights,biases)
loss = mse(predictions,targets)
print(loss)

tensor(31086.9570, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 200 epochs.

In [25]:
#Train for 200 epochs
for i in range(200):
    predictions = model(train_data, weights, biases)
    loss = mse(predictions, targets)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

Once again, let's verify that the loss is now lower:

In [26]:
#Calculate loss
predictions = model(train_data, weights, biases)
loss = mse(predictions, targets)
print(loss)

tensor(61.0709, grad_fn=<DivBackward0>)


In [28]:
#Print predictions
print(predictions)

tensor([[ 58.7025,  73.2683],
        [ 83.8416, 101.1629],
        [112.5057, 127.0758],
        [ 29.4414,  55.4837],
        [100.0147, 108.9294]], grad_fn=<AddBackward0>)


In [29]:
#print targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
